# Apartado 1 — Explicación del conjunto de datos (T1D Guertin 2024)

## Objetivo
- Identificar la temática del dataset y su interés analítico.
- Confirmar los dos archivos de trabajo: `survey_data_and_results_final.xlsx` y `assay_final_genotyping_file.xlsx`.
- Mostrar tamaño, estructura y relación entre tablas mediante una clave común.
- Declarar explícitamente que en genotipado el valor `-9` representa missing.

## Contexto mínimo (sin inventar resultados)
- Tema: cribado de riesgo de diabetes tipo 1 (T1D Screening Project, Guertin 2024).
- Tablas objetivo para análisis relacional: survey y genotyping.
- Clave prevista de JOIN: `SUBJECT_ID` (o equivalente tras ver columnas reales).

In [1]:
!git clone https://github.com/kachytronico/BDA_dataset

Cloning into 'BDA_dataset'...


In [ ]:
import pandas as pd
from pathlib import Path
import os

# 1) Rutas base (repo clonado)
REPO = Path("/content/BDA_dataset")
SRC_DIR = REPO / "Dataset not incorporated into the T1DKP"   # <- RUTA REAL SEGÚN README
RAW_DIR = Path("/content/data/raw")
CSV_DIR = Path("/content/data/csv")
RAW_DIR.mkdir(parents=True, exist_ok=True)
CSV_DIR.mkdir(parents=True, exist_ok=True)

assert SRC_DIR.exists(), f"No existe la carpeta fuente esperada: {SRC_DIR}"

# 2) Rutas a los XLSX (dataset Guertin 2024)
XLS_SURVEY = SRC_DIR / "survey_data_and_results_final.xlsx"
XLS_GENO   = SRC_DIR / "assay_final_genotyping_file.xlsx"

assert XLS_SURVEY.exists(), f"No existe: {XLS_SURVEY}"
assert XLS_GENO.exists(), f"No existe: {XLS_GENO}"

# 3) Copia RAW (evidencia + mantener originales intactos)
RAW_SURVEY = RAW_DIR / XLS_SURVEY.name
RAW_GENO   = RAW_DIR / XLS_GENO.name
RAW_SURVEY.write_bytes(XLS_SURVEY.read_bytes())
RAW_GENO.write_bytes(XLS_GENO.read_bytes())

print("✅ RAW copiados en:", RAW_DIR)
print("📁 Contenido RAW:")
!ls -lh /content/data/raw

# 4) Leer Excel y convertir a CSV (formato requerido por Pig/Spark/Hive)
df_survey = pd.read_excel(RAW_SURVEY)
df_geno   = pd.read_excel(RAW_GENO)

CSV_SURVEY = CSV_DIR / "survey.csv"
CSV_GENO   = CSV_DIR / "genotyping.csv"

df_survey.to_csv(CSV_SURVEY, index=False)
df_geno.to_csv(CSV_GENO, index=False)

print(f"✅ survey.csv guardado: {df_survey.shape[0]} filas, {df_survey.shape[1]} columnas")
print(f"✅ genotyping.csv guardado: {df_geno.shape[0]} filas, {df_geno.shape[1]} columnas")

# 5) Evidencias mínimas
print("\n📁 CSV generados en /content/data/csv/:")
!ls -lh /content/data/csv

print("\n📊 Preview survey (5 filas):")
display(df_survey.head())

print("\n📊 Preview genotyping (5 filas):")
display(df_geno.head())


In [ ]:
from pathlib import Path
import pandas as pd

survey_name = 'survey_data_and_results_final.xlsx'
geno_name = 'assay_final_genotyping_file.xlsx'

def find_file(filename):
    roots = [Path.cwd(), Path('/content')]
    for root in roots:
        if root.exists():
            direct = root / filename
            if direct.exists():
                return direct
            matches = list(root.rglob(filename))
            if matches:
                return matches[0]
    return None

survey_path = find_file(survey_name)
geno_path = find_file(geno_name)

evidence = pd.DataFrame({
    'archivo': [survey_name, geno_name],
    'encontrado': [survey_path is not None, geno_path is not None],
    'ruta': [str(survey_path) if survey_path else 'NO_ENCONTRADO',
             str(geno_path) if geno_path else 'NO_ENCONTRADO']
})
display(evidence)

if survey_path and geno_path:
    df_survey = pd.read_excel(survey_path)
    df_geno = pd.read_excel(geno_path)

    print('survey shape:', df_survey.shape)
    print('genotyping shape:', df_geno.shape)

    common_cols = sorted(set(df_survey.columns).intersection(set(df_geno.columns)))
    print('columnas comunes (candidatas a JOIN):', common_cols[:20])

    if 'SUBJECT_ID' in common_cols:
        print('SUBJECT_ID detectado como clave de relación candidata.')

    print('\nMuestra survey:')
    display(df_survey.head(3))
    print('\nMuestra genotyping:')
    display(df_geno.head(3))

    missing_neg9 = (df_geno == -9).sum(numeric_only=False).sum()
    print('Total de celdas con -9 en genotyping (missing codificado):', int(missing_neg9))
else:
    print('No se encontraron ambos XLSX en el entorno actual.')
    print('Este apartado queda preparado; la evidencia completa aparecerá cuando estén disponibles los archivos.')

## Evidencia visible mínima (Apartado 1)
- Tabla con estado de los 2 archivos requeridos (encontrado/ruta).
- Tamaño de `survey` y `genotyping` (`shape`) cuando los archivos estén presentes.
- Lista de columnas comunes para justificar la relación entre tablas.
- Muestras `head(3)` de ambas tablas.
- Conteo de `-9` en genotipado como evidencia de missing codificado.

### Conclusiones (completar tras ejecutar)
- [Pendiente de ejecución] Confirmación de disponibilidad de ambos archivos.
- [Pendiente de ejecución] Clave de JOIN validada en datos reales.
- [Pendiente de ejecución] Estado inicial de calidad ligado al valor `-9`.